# Call Context Summary

## 1. Setup Workspace

In [1]:
import json
import pathlib
import copy
import operator
import pathlib
import os

import anytree
import networkx as nx
import sympy

import paptree

## 2. Load Data

The fibonnaci library contains fours implementations of a fibonacci generator:

1. Recursive (naive)
2. Recursive (memoized)
3. Iterative
4. Lookup table

The library unit tests verify the first eight fibonnaci numbers returned by each implementation as well as the behavior when the user requests a fibonnaci number to large to fit in the return type. This should result in 36 (`4 * (8 + 1)`) traces.

In [2]:
trace_file = pathlib.Path.cwd().parent / "data/fibonacci/paptrace.json"
trees = paptree.utils.from_file(trace_file)
print(f"Loaded {len(trees)} traces.")

Loaded 36 traces.


## 3. Node Summary

### 3.1 Unfiltered Nodes

In [3]:
nodes = []
for tree in trees:
    nodes.extend(anytree.PreOrderIter(tree.root))
print(f"Node count: {len(nodes)}")

Node count: 4974


In [4]:
node_types = {}
for node in nodes:
    node_types.setdefault(node.type, []).append(node)
print(f"Node type count: {len(node_types)}")

Node type count: 7


In [5]:
print("Nodes per type:")
for k, v in node_types.items():
    print(f"- {k}: {len(v)}")

Nodes per type:
- CalleeExpr: 1354
- IfThenStmt: 923
- ReturnStmt: 1350
- CallerExpr: 1316
- CXXThrowExpr: 4
- ForStmt: 6
- LoopIter: 21


### 3.2. Unique Nodes

In [6]:
unique_nodes = []
for node in nodes:
    if node not in unique_nodes:
        unique_nodes.append(node)
print(f"Unique node count: {len(unique_nodes)}")

Unique node count: 326


In [7]:
# We expect this count to match the unfiltered node type count.
unique_node_types = {}
for node in unique_nodes:
    unique_node_types.setdefault(node.type, []).append(node)
print(f"Node type count: {len(unique_node_types)}")

Node type count: 7


In [8]:
print("Nodes per type:")
for k, v in unique_node_types.items():
    print(f"- {k}: {len(v)}")

Nodes per type:
- CalleeExpr: 101
- IfThenStmt: 7
- ReturnStmt: 7
- CallerExpr: 205
- CXXThrowExpr: 4
- ForStmt: 1
- LoopIter: 1


We can reason about the control flow related unique node counts using the source:

- The source has 7 `if` statements (`grep if src/fibonacci.cpp | wc -l`).
    - Note: This includes `else if`.
- The source has 7 `return` statements (`grep return src/fibonacci.cpp | wc -l`).
- The source has 4 `throw` statements (`grep throw src/fibonacci.cpp | wc -l`).
- The source has 1 `for` statement (`grep "for " src/fibonacci.cpp | wc -l`).

## 4. Trace Summary

As mentioned in Section 2, we expect 9 traces for each of the 4 exposed library functions for a total of 36 traces.

In [9]:
binned_trees = {}
for tree in trees:
    binned_trees.setdefault(tree.root.sig, []).append(tree)
print(f"Trace entry point count: {len(binned_trees)}")

Trace entry point count: 4


In [10]:
print("Traces per entry point:")
for k, v in binned_trees.items():
    print(f"- {k}: {len(v)}")

Traces per entry point:
- unsigned long long fibonacci::RecursiveNaive(unsigned short): 9
- unsigned long long fibonacci::RecursiveMemo(unsigned short): 9
- unsigned long long fibonacci::Iterative(unsigned short): 9
- unsigned long long fibonacci::LookupTable(unsigned short): 9


## 5. Trace Analysis

### 5.1. Recursive (Naive)

In [11]:
rec_naive_sig = f"unsigned long long fibonacci::RecursiveNaive(unsigned short)"
rec_naive_trees = binned_trees[rec_naive_sig]

def to_params_str(params):
    #return ", ".join([f"{param['name']}={param['value']}" for param in params])
    return ", ".join([f"{param['value']}" for param in params])

def to_call_str(node):
    return f"{node.sig}: ({to_params_str(node.params)})"

print("Recorded calls:")
for tree in rec_naive_trees:
    print(f"{to_call_str(tree.root)}")

Recorded calls:
unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (2)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (3)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (4)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (5)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (6)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (7)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (94)


In [12]:
# Peek at select traces that we know take differing paths.
def to_simple_node_view(node):
    sym = "sym @ " if hasattr(node, "target") else ""
    if paptree.Node.is_call_type(node.type):
        desc = f" {to_call_str(node)}"
    else:
        desc = f" {node.type}: {node.desc}"
    return f"({sym}{node.name}){desc}"

for tree in operator.itemgetter(0, 2, 8)(rec_naive_trees):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
└── (2106009) IfThenStmt: n < 2
    └── (2106007) ReturnStmt: return n

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (2)
└── (2106188) ReturnStmt: return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)
    ├── (2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
    │   └── (2106009) IfThenStmt: n < 2
    │       └── (2106007) ReturnStmt: return n
    ├── (2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
    │   └── (2106009) IfThenStmt: n < 2
    │       └── (2106007) ReturnStmt: return n
    └── (2106184) unsigned long long operator+(unsigned long long, unsigned long long): (1, 0)
        └── (2106117) int operator-(unsigned short, int): (2, 1)
            ├── (2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
            │   └── (2106009) IfThenStmt: n < 2
            │       └── (2106007) ReturnStmt: return n


In [13]:
# Let's eliminate the children of the operator nodes. They are there by mistake.
for tree in trees:
    for node in anytree.PreOrderIter(tree.root):
        if node.type == "CallerExpr":
            if "operator+" in node.sig or "operator-" in node.sig or "operator=" in node.sig:
                node.children = ()

for tree in operator.itemgetter(0, 2, 8)(rec_naive_trees):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
└── (2106009) IfThenStmt: n < 2
    └── (2106007) ReturnStmt: return n

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (2)
└── (2106188) ReturnStmt: return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)
    ├── (2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
    │   └── (2106009) IfThenStmt: n < 2
    │       └── (2106007) ReturnStmt: return n
    ├── (2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
    │   └── (2106009) IfThenStmt: n < 2
    │       └── (2106007) ReturnStmt: return n
    └── (2106184) unsigned long long operator+(unsigned long long, unsigned long long): (1, 0)

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (94)
└── (2106081) IfThenStmt: n > 93
    └── (2106075) CXXThrowExpr: throw std::overflow_error("n must be less than 94")



In [14]:
def is_cf_node(node):
    return node.type in ["IfThenStmt", "ReturnStmt", "CXXThrowExpr", "ForStmt", "LoopIter"]

# NOTE: We currently do not need to account for the context of calls made inside the trace
# because we have not "squashed" those call nodes. If we did "squash" call nodes, then we
# would need to query the path for that call to be able to correctly form partitions.
fn_paths = {}
for tree in operator.itemgetter(0, 2, 3, 8)(rec_naive_trees):
    #print(f"\n{fn_name}({to_params_str(tree.root.params)})")
    cf_nodes = [node.name for node in anytree.PreOrderIter(tree.root, filter_=is_cf_node)]
    fn_paths.setdefault(tuple(cf_nodes), []).append(tree)
    #print(anytree.RenderTree(tree), "\n")

# Q. What do we expect here?
#
# A1. We expect 3 paths. Each trace in n > 1 =< 93 takes Path 2. Maximum of 3 paths.
# Path 1: n <= 1
# Path 2: n > 1 =< 93
# Path 3: n > 93
#
# A2. We expect 4 paths. Each trace in n > 1 =< 93 takes its own Path. Maximum of 94 paths.
# Path 1: n <= 1
# Path 2: n == 2
# Path 3: n > 93
# 
# Clearly A1 is preferable to A2, but our current code produces A2.

print(f"Path count: {len(fn_paths)}", "\n")
print("Traces per path:")
for k, v in fn_paths.items():
    print(f"- {k}: {len(v)}")

Path count: 4 

Traces per path:
- (2106009, 2106007): 1
- (2106188, 2106009, 2106007, 2106009, 2106007): 1
- (2106188, 2106188, 2106009, 2106007, 2106009, 2106007, 2106009, 2106007): 1
- (2106081, 2106075): 1


In [15]:
# Getting to A2.
#
# 1. First thing we can do is "squash" calls so that recursion does not result in differing paths.
# 2. Next, we can use the expression for the "squashed" calls to differentiate between calls to different paths.
# 2a. For now, we can just fake an expression for the "squashed" calls.
# 2b. In practice, we will need to solve for the squashed calls using symbolic regression.

# STEP 1
# Let's start by putting our trace root nodes in a container that allows us
# to lookup the trace by context.
# NOTE: This does not account for complete non-root traces (e.g., RecursiveMemoImpl).
trace_roots = {}
for tree in trees:
    call_str = to_call_str(tree.root)
    if call_str not in trace_roots:
        trace_roots[call_str] = tree.root
    else:
        if tree.root != trace_roots[call_str]:
            raise RuntimeError(
                "Unhandled scenario: 2 traces with the same context have differing trees.")
            
# Now we walk the trees looking for child nodes that can be substituted with a
# trace root node.
for tree in trees:
    for node in anytree.PreOrderIter(tree.root):
        if node.is_root:
            continue
        replace_children = False
        new_children = []
        for child in node.children:
            if child.type == "CalleeExpr":
                child_call_str = to_call_str(child)
                if child_call_str in trace_roots:
                    replace_children = True
                    child = anytree.SymlinkNode(child)
            new_children.append(child)
        if replace_children:
            node.children = tuple(new_children)
            
# Peek again at the select traces now that calls have been "squashed".
for tree in operator.itemgetter(0, 2, 8)(rec_naive_trees):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
└── (2106009) IfThenStmt: n < 2
    └── (2106007) ReturnStmt: return n

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (2)
└── (2106188) ReturnStmt: return RecursiveNaive(n - 1) + RecursiveNaive(n - 2)
    ├── (sym @ 2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
    ├── (sym @ 2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
    └── (2106184) unsigned long long operator+(unsigned long long, unsigned long long): (1, 0)

(2106190) unsigned long long fibonacci::RecursiveNaive(unsigned short): (94)
└── (2106081) IfThenStmt: n > 93
    └── (2106075) CXXThrowExpr: throw std::overflow_error("n must be less than 94")



In [16]:
# Let's group by path again. We expect to have a path count of 3.
fn_paths = {}
for tree in rec_naive_trees:
    cf_nodes = [node.name for node in anytree.PreOrderIter(tree.root, filter_=is_cf_node)]
    fn_paths.setdefault(tuple(cf_nodes), []).append(tree)

print(f"Path count: {len(fn_paths)}", "\n")
print("Traces per path:")
for k, v in fn_paths.items():
    print(f"- {k}: {len(v)}")

Path count: 3 

Traces per path:
- (2106009, 2106007): 2
- (2106188,): 6
- (2106081, 2106075): 1


In [17]:
# STEP 2
#
# At this point we've only performed partial path partitioning. For example, RecursiveNaive(2)
# and RecursiveNaive(3) seem to take the same path, but we don't know until we've solved for
# the complexity of their recursive calls.
# 
# RN(2) depends on both RN(1) and RN(0)
# RN(3) depends on both RN(2) and RN(1)
#
# This means these relationships can be modeled by the dependency tree:
# RN(3)
# + RN(2)
#   + RN(1)
#   + RN(0)
#
# We need to generate this dependency tree using code.

G = nx.DiGraph()
G.add_node("root")
for tree in rec_naive_trees:
    nodes = [node for node in anytree.PreOrderIter(tree.root, filter_=lambda n: hasattr(n, "target"))]
    target_str = to_call_str(tree.root)
    if not nodes:
        G.add_edge(target_str, "root")
    else:
        for node in nodes:
            prereq_str = to_call_str(node)
            G.add_edge(prereq_str, target_str)

dfs_postorder = [node for node in nx.dfs_postorder_nodes(G)]
eval_order = dfs_postorder[::-1]

print("\nEval Order:")
for x in eval_order:
    print(x)


Eval Order:
unsigned long long fibonacci::RecursiveNaive(unsigned short): (94)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (1)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (0)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (2)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (3)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (4)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (5)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (6)
unsigned long long fibonacci::RecursiveNaive(unsigned short): (7)
root


In [18]:
# Let's structure traces for the solver.

# Convert path tuples to IDs.
# We store an ID->trees mapping so that we can easily query trees for a path ID.
path_ids = {}
id_paths = {}
for path_tuple, trees in fn_paths.items():
    path_id = path_ids.setdefault(path_tuple, len(path_ids))
    id_paths[path_id] = trees

# It is expected that we start with some known values.
# NOTE: This currently uses signatures to generalize (i.e., it is not accounting for context).
known = {
    "int operator-(unsigned short, int)": [""], 
    "unsigned long long operator+(unsigned long long, unsigned long long)": [""],
    "unsigned long long operator=(unsigned long long, unsigned long long)": [""],
}
exprs = {}
call_path_ids = {}

# Expr data should map: sig->path_id->ctx->expr
expr_data = {}

def to_expr(node, level=0):
    subexpr = []
    for child in node.children:
        subexpr.extend(to_expr(child, level+1))
    if node.is_root:
        subexpr.append(f"C_F{node.name}_P{call_path_ids[to_call_str(node)]}")
    elif paptree.Node.is_call_type(node.type):
        #print(f"{' ' * 2 * level}including node {to_simple_node_view(node)}")
        if node.sig in known:
            subexpr.extend(known[node.sig])
        else:
            subexpr.extend(exprs[to_call_str(node)])
    else:
        #print(f"{' ' * 2 * level}excluding node {to_simple_node_view(node)}")
        pass
    return subexpr

for path_id, path_trees in id_paths.items():
    for tree in path_trees:
        call_path_ids[to_call_str(tree.root)] = path_id
        expr = to_expr(tree.root)
        exprs[to_call_str(tree.root)] = expr
        
        sig_data = expr_data.setdefault(tree.root.sig, {})
        path_data = sig_data.setdefault(f"path_{path_id}", {})
        param_str = to_params_str(tree.root.params)
        sympy_expr = sympy.sympify(' + '.join(expr))
        path_data[param_str] = sympy.srepr(sympy_expr)

print(json.dumps(expr_data[rec_naive_sig], indent=2))

{
  "path_0": {
    "0": "Symbol('C_F2106190_P0')",
    "1": "Symbol('C_F2106190_P0')"
  },
  "path_1": {
    "2": "Add(Mul(Integer(2), Symbol('C_F2106190_P0')), Symbol('C_F2106190_P1'))",
    "3": "Add(Mul(Integer(3), Symbol('C_F2106190_P0')), Mul(Integer(2), Symbol('C_F2106190_P1')))",
    "4": "Add(Mul(Integer(5), Symbol('C_F2106190_P0')), Mul(Integer(4), Symbol('C_F2106190_P1')))",
    "5": "Add(Mul(Integer(8), Symbol('C_F2106190_P0')), Mul(Integer(7), Symbol('C_F2106190_P1')))",
    "6": "Add(Mul(Integer(13), Symbol('C_F2106190_P0')), Mul(Integer(12), Symbol('C_F2106190_P1')))",
    "7": "Add(Mul(Integer(21), Symbol('C_F2106190_P0')), Mul(Integer(20), Symbol('C_F2106190_P1')))"
  },
  "path_2": {
    "94": "Symbol('C_F2106190_P2')"
  }
}


### 5.2. Iterative

In [19]:
iter_sig = f"unsigned long long fibonacci::Iterative(unsigned short)"
iter_trees = binned_trees[iter_sig]

print("Recorded calls:")
for tree in iter_trees:
    print(f"{to_call_str(tree.root)}")

Recorded calls:
unsigned long long fibonacci::Iterative(unsigned short): (0)
unsigned long long fibonacci::Iterative(unsigned short): (1)
unsigned long long fibonacci::Iterative(unsigned short): (2)
unsigned long long fibonacci::Iterative(unsigned short): (3)
unsigned long long fibonacci::Iterative(unsigned short): (4)
unsigned long long fibonacci::Iterative(unsigned short): (5)
unsigned long long fibonacci::Iterative(unsigned short): (6)
unsigned long long fibonacci::Iterative(unsigned short): (7)
unsigned long long fibonacci::Iterative(unsigned short): (94)


In [20]:
# Peek at select traces that we know take differing paths.
for tree in operator.itemgetter(0, 2, 8)(iter_trees):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(2110045) unsigned long long fibonacci::Iterative(unsigned short): (0)
└── (2109766) IfThenStmt: n < 2
    └── (2109764) ReturnStmt: return n

(2110045) unsigned long long fibonacci::Iterative(unsigned short): (2)
├── (2110029) ForStmt: for (unsigned short i = 2; i <= n; ++i)
│   └── (2110029) LoopIter: LoopIter
│       ├── (2109990) unsigned long long operator=(unsigned long long, unsigned long long): (105553123975920, 1)
│       ├── (2110005) unsigned long long operator=(unsigned long long, unsigned long long): (0, 1)
│       └── (2110020) unsigned long long operator=(unsigned long long, unsigned long long): (1, 1)
└── (2110043) ReturnStmt: return fib

(2110045) unsigned long long fibonacci::Iterative(unsigned short): (94)
└── (2109827) IfThenStmt: n > 93
    └── (2109821) CXXThrowExpr: throw std::overflow_error("n must be less than 94")



In [21]:
# Let's group by path again. We expect to have a path count of 3.
fn_paths = {}
for tree in iter_trees:
    cf_nodes = [node.name for node in anytree.PreOrderIter(tree.root, filter_=is_cf_node)]
    fn_paths.setdefault(tuple(cf_nodes), []).append(tree)

print(f"Path count: {len(fn_paths)}", "\n")
print("Traces per path:")
for k, v in fn_paths.items():
    print(f"- {k}: {len(v)}")

Path count: 8 

Traces per path:
- (2109766, 2109764): 2
- (2110029, 2110029, 2110043): 1
- (2110029, 2110029, 2110029, 2110043): 1
- (2110029, 2110029, 2110029, 2110029, 2110043): 1
- (2110029, 2110029, 2110029, 2110029, 2110029, 2110043): 1
- (2110029, 2110029, 2110029, 2110029, 2110029, 2110029, 2110043): 1
- (2110029, 2110029, 2110029, 2110029, 2110029, 2110029, 2110029, 2110043): 1
- (2109827, 2109821): 1


In [22]:
G = nx.DiGraph()
G.add_node("root")
for tree in iter_trees:
    nodes = [node for node in anytree.PreOrderIter(tree.root, filter_=lambda n: hasattr(n, "target"))]
    target_str = to_call_str(tree.root)
    if not nodes:
        G.add_edge(target_str, "root")
    else:
        for node in nodes:
            prereq_str = to_call_str(node)
            G.add_edge(prereq_str, target_str)

dfs_postorder = [node for node in nx.dfs_postorder_nodes(G)]
eval_order = dfs_postorder[::-1]

print("\nEval Order:")
for x in eval_order:
    print(x)


Eval Order:
unsigned long long fibonacci::Iterative(unsigned short): (94)
unsigned long long fibonacci::Iterative(unsigned short): (7)
unsigned long long fibonacci::Iterative(unsigned short): (6)
unsigned long long fibonacci::Iterative(unsigned short): (5)
unsigned long long fibonacci::Iterative(unsigned short): (4)
unsigned long long fibonacci::Iterative(unsigned short): (3)
unsigned long long fibonacci::Iterative(unsigned short): (2)
unsigned long long fibonacci::Iterative(unsigned short): (1)
unsigned long long fibonacci::Iterative(unsigned short): (0)
root


In [23]:
# Let's structure traces for the solver.

# Convert path tuples to IDs.
# We store an ID->trees mapping so that we can easily query trees for a path ID.
path_ids = {}
id_paths = {}
for path_tuple, trees in fn_paths.items():
    path_id = path_ids.setdefault(path_tuple, len(path_ids))
    id_paths[path_id] = trees

for path_id, path_trees in id_paths.items():
    for tree in path_trees:
        call_path_ids[to_call_str(tree.root)] = path_id
        expr = to_expr(tree.root)
        exprs[to_call_str(tree.root)] = expr
        
        sig_data = expr_data.setdefault(tree.root.sig, {})
        path_data = sig_data.setdefault(f"path_{path_id}", {})
        param_str = to_params_str(tree.root.params)
        with sympy.evaluate(False):
            sympy_expr = sympy.sympify(' + '.join(expr))
            path_data[param_str] = sympy.srepr(sympy_expr)

print(json.dumps(expr_data[iter_sig], indent=2))

{
  "path_0": {
    "0": "Symbol('C_F2110045_P0')",
    "1": "Symbol('C_F2110045_P0')"
  },
  "path_1": {
    "2": "Symbol('C_F2110045_P1')"
  },
  "path_2": {
    "3": "Symbol('C_F2110045_P2')"
  },
  "path_3": {
    "4": "Symbol('C_F2110045_P3')"
  },
  "path_4": {
    "5": "Symbol('C_F2110045_P4')"
  },
  "path_5": {
    "6": "Symbol('C_F2110045_P5')"
  },
  "path_6": {
    "7": "Symbol('C_F2110045_P6')"
  },
  "path_7": {
    "94": "Symbol('C_F2110045_P7')"
  }
}


### 5.3. Lookup Table

In [24]:
lookup_sig = f"unsigned long long fibonacci::LookupTable(unsigned short)"
lookup_trees = binned_trees[lookup_sig]

print("Recorded calls:")
for tree in lookup_trees:
    print(f"{to_call_str(tree.root)}")

Recorded calls:
unsigned long long fibonacci::LookupTable(unsigned short): (0)
unsigned long long fibonacci::LookupTable(unsigned short): (1)
unsigned long long fibonacci::LookupTable(unsigned short): (2)
unsigned long long fibonacci::LookupTable(unsigned short): (3)
unsigned long long fibonacci::LookupTable(unsigned short): (4)
unsigned long long fibonacci::LookupTable(unsigned short): (5)
unsigned long long fibonacci::LookupTable(unsigned short): (6)
unsigned long long fibonacci::LookupTable(unsigned short): (7)
unsigned long long fibonacci::LookupTable(unsigned short): (94)


In [25]:
# Peek at select traces that we know take differing paths.
# Note: Unlike preview implementations, calls for number 0 and 2 take the same path.
for tree in operator.itemgetter(0, 2, 8)(lookup_trees):
    for pre, _, node in anytree.RenderTree(tree.root):
        print(f"{pre}{to_simple_node_view(node)}")
    print()

(2110185) unsigned long long fibonacci::LookupTable(unsigned short): (0)
└── (2110183) ReturnStmt: return s_fib_table[n]

(2110185) unsigned long long fibonacci::LookupTable(unsigned short): (2)
└── (2110183) ReturnStmt: return s_fib_table[n]

(2110185) unsigned long long fibonacci::LookupTable(unsigned short): (94)
└── (2110155) IfThenStmt: n > 93
    └── (2110149) CXXThrowExpr: throw std::overflow_error("n must be less than 94")



In [26]:
# Let's group by path again. We expect to have a path count of 2.
fn_paths = {}
for tree in lookup_trees:
    cf_nodes = [node.name for node in anytree.PreOrderIter(tree.root, filter_=is_cf_node)]
    fn_paths.setdefault(tuple(cf_nodes), []).append(tree)

print(f"Path count: {len(fn_paths)}", "\n")
print("Traces per path:")
for k, v in fn_paths.items():
    print(f"- {k}: {len(v)}")

Path count: 2 

Traces per path:
- (2110183,): 8
- (2110155, 2110149): 1


In [27]:
G = nx.DiGraph()
G.add_node("root")
for tree in lookup_trees:
    nodes = [node for node in anytree.PreOrderIter(tree.root, filter_=lambda n: hasattr(n, "target"))]
    target_str = to_call_str(tree.root)
    if not nodes:
        G.add_edge(target_str, "root")
    else:
        for node in nodes:
            prereq_str = to_call_str(node)
            G.add_edge(prereq_str, target_str)

dfs_postorder = [node for node in nx.dfs_postorder_nodes(G)]
eval_order = dfs_postorder[::-1]

print("\nEval Order:")
for x in eval_order:
    print(x)


Eval Order:
unsigned long long fibonacci::LookupTable(unsigned short): (94)
unsigned long long fibonacci::LookupTable(unsigned short): (7)
unsigned long long fibonacci::LookupTable(unsigned short): (6)
unsigned long long fibonacci::LookupTable(unsigned short): (5)
unsigned long long fibonacci::LookupTable(unsigned short): (4)
unsigned long long fibonacci::LookupTable(unsigned short): (3)
unsigned long long fibonacci::LookupTable(unsigned short): (2)
unsigned long long fibonacci::LookupTable(unsigned short): (1)
unsigned long long fibonacci::LookupTable(unsigned short): (0)
root


In [28]:
# Convert path tuples to IDs.
# We store an ID->trees mapping so that we can easily query trees for a path ID.
path_ids = {}
id_paths = {}
for path_tuple, trees in fn_paths.items():
    path_id = path_ids.setdefault(path_tuple, len(path_ids))
    id_paths[path_id] = trees

# It is expected that we start with some known values.
# NOTE: This currently uses signatures to generalize (i.e., it is not accounting for context).
known = {
    "int operator-(unsigned short, int)": [""], 
    "unsigned long long operator+(unsigned long long, unsigned long long)": [""],
}
exprs = {}
call_path_ids = {}

for path_id, path_trees in id_paths.items():
    for tree in path_trees:
        call_path_ids[to_call_str(tree.root)] = path_id
        expr = to_expr(tree.root)
        exprs[to_call_str(tree.root)] = expr
        
        sig_data = expr_data.setdefault(tree.root.sig, {})
        path_data = sig_data.setdefault(f"path_{path_id}", {})
        param_str = to_params_str(tree.root.params)
        sympy_expr = sympy.sympify(' + '.join(expr))
        path_data[param_str] = sympy.srepr(sympy_expr)

print(json.dumps(expr_data[lookup_sig], indent=2))

{
  "path_0": {
    "0": "Symbol('C_F2110185_P0')",
    "1": "Symbol('C_F2110185_P0')",
    "2": "Symbol('C_F2110185_P0')",
    "3": "Symbol('C_F2110185_P0')",
    "4": "Symbol('C_F2110185_P0')",
    "5": "Symbol('C_F2110185_P0')",
    "6": "Symbol('C_F2110185_P0')",
    "7": "Symbol('C_F2110185_P0')"
  },
  "path_1": {
    "94": "Symbol('C_F2110185_P1')"
  }
}


## 6. Expression Data

Let's take a look at the expression data we have generated from the traces.

In [29]:
print(json.dumps(expr_data, indent=2))

{
  "unsigned long long fibonacci::RecursiveNaive(unsigned short)": {
    "path_0": {
      "0": "Symbol('C_F2106190_P0')",
      "1": "Symbol('C_F2106190_P0')"
    },
    "path_1": {
      "2": "Add(Mul(Integer(2), Symbol('C_F2106190_P0')), Symbol('C_F2106190_P1'))",
      "3": "Add(Mul(Integer(3), Symbol('C_F2106190_P0')), Mul(Integer(2), Symbol('C_F2106190_P1')))",
      "4": "Add(Mul(Integer(5), Symbol('C_F2106190_P0')), Mul(Integer(4), Symbol('C_F2106190_P1')))",
      "5": "Add(Mul(Integer(8), Symbol('C_F2106190_P0')), Mul(Integer(7), Symbol('C_F2106190_P1')))",
      "6": "Add(Mul(Integer(13), Symbol('C_F2106190_P0')), Mul(Integer(12), Symbol('C_F2106190_P1')))",
      "7": "Add(Mul(Integer(21), Symbol('C_F2106190_P0')), Mul(Integer(20), Symbol('C_F2106190_P1')))"
    },
    "path_2": {
      "94": "Symbol('C_F2106190_P2')"
    }
  },
  "unsigned long long fibonacci::Iterative(unsigned short)": {
    "path_0": {
      "0": "Symbol('C_F2110045_P0')",
      "1": "Symbol('C_F211004

In [30]:
fib_data_path = pathlib.Path(os.path.abspath("")).parent / "data" / "fibonacci"
with open(fib_data_path / "paptrace_expr_data.json", "w") as f_out:
    f_out.write(json.dumps(expr_data, indent=4))